In [ ]:
# import cPickle as pickle
import ast
import pickle
import re

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import json

In [ ]:
import findspark

findspark.init()

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession


spark = SparkSession\
        .builder\
        .appName("TrainingOnCombinedSummary")\
        .config('spark.sql.warehouse.dir','file:///home/sudipto21048867/Pyspark/spark-warehouse')\
        .config("spark.sql.caseSensitive","true")\
        .getOrCreate()

In [ ]:
#data = json.load(open('C:/Users/sudsaha1/Documents/My Received Files/TransformedSummary.json'))

#TransactionDf_pd = [item for items in data.values() for item in items]
#TransactionDf_pd=pd.DataFrame(TransactionDf_pd) 


#TransactionDf_pd.head()
TransactionDf_pd = pd.read_csv('C:/Users/sudsaha1/Documents/My Received Files/CombinedSummary.csv')

In [ ]:
TransactionDf=spark.createDataFrame(TransactionDf_pd)
pd.DataFrame(TransactionDf.take(5),columns=TransactionDf.columns).transpose()

In [ ]:
TransactionDf_pd.dtypes

In [7]:
#sc = spark.sparkContext
#Transaction_rdd = sc.textFile("D:/Llyods/BASE_Data/CombinedSummary.csv")
#Transaction_rdd.take(5)
TransactionDf.dtypes

In [8]:
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from pyspark.sql.functions import UserDefinedFunction as udf
from pyspark.sql.functions import expr
from pyspark.sql import SQLContext, Row

In [9]:
TransactionDf_pd['AmountInGBP'] = TransactionDf_pd['Amount'].map(lambda x: ast.literal_eval(x)['Amount']).astype(np.float)
TransactionDf_pd['Type'] = TransactionDf_pd['BankTransactionCode'].map(lambda x: ast.literal_eval(x[:-1] + "\"\"" + '}')['Code'])

In [10]:
TransactionDf_pd.head()

,Unnamed: 0,AccountId,Amount,BankTransactionCode,BookingDateTime,CreditDebitIndicator,Status,TransactionId,TransactionInformation,ValueDateTime,category,Detailed Category,AmountInGBP,Type
0,296,32353546,"{""Amount"": ""139.9"", ""Currency"": ""GBP""}","{""Code"": ""CHP"","" SubCode"": }",2016-11-29T20:21:50-08:00,Credit,Booked,100223,"/RFB/GMCI STATMT , PCM55CI72578736 , HENNER GMC",2016-11-29T20:21:50-08:00,Services,Health,139.9,CHP
1,76,32353530,"{""Amount"": ""388.5"", ""Currency"": ""GBP""}","{""Code"": ""CHP"","" SubCode"": }",2016-12-05T20:21:50-08:00,Credit,Booked,100315,"/RFB/GMCI STATMT , PCM55CI72684987 , HENNER GMC",2016-12-05T20:21:50-08:00,Services,Health,388.5,CHP
2,82,32353542,"{""Amount"": ""50.4"", ""Currency"": ""GBP""}","{""Code"": ""CHP"","" SubCode"": }",2016-12-06T20:21:50-08:00,Credit,Booked,100321,"/RFB/GMCI STATMT , PCM55CI72706905 , HENNER GMC",2016-12-06T20:21:50-08:00,Services,Health,50.4,CHP
3,268,32353522,"{""Amount"": ""404"", ""Currency"": ""GBP""}","{""Code"": ""CHP"","" SubCode"": }",2017-01-11T20:21:50-08:00,Credit,Booked,100401,"/RFB/GMCI STATMT , PCM55CI73243020 , HENNER GMC",2017-01-11T20:21:50-08:00,Services,Health,404.0,CHP
4,269,32353524,"{""Amount"": ""45"", ""Currency"": ""GBP""}","{""Code"": ""CHP"","" SubCode"": }",2017-01-24T20:21:50-08:00,Credit,Booked,100402,"/RFB/GMCI STATMT , PCM55CI73446179 , HENNER GMC",2017-01-24T20:21:50-08:00,Services,Health,45.0,CHP


In [11]:
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from pyspark.sql.functions import UserDefinedFunction as udf

In [12]:

AmountInGBP = udf(lambda x: (x.split(",")[0].split(":")[1]).replace('"', '').strip() , StringType())

Type = udf(lambda x: (x.split(",")[0].split(":")[1]).replace('"', '').strip() , StringType())

def string_to_float(x):
    return float(x)

udfstring_to_float = udf(string_to_float, FloatType())
#paid_out_cleaned = udf(lambda y: y.replace(',', '') if y!='-' else '0', StringType())

In [13]:

TransactionDf = TransactionDf.withColumn('AmountInGBP',udfstring_to_float(AmountInGBP(TransactionDf['Amount']))).\
                              withColumn('Type',Type(TransactionDf['BankTransactionCode']))

In [14]:
TransactionDf.printSchema()

root
 |-- Unnamed: 0: long (nullable = true)
 |-- AccountId: long (nullable = true)
 |-- Amount: string (nullable = true)
 |-- BankTransactionCode: string (nullable = true)
 |-- BookingDateTime: string (nullable = true)
 |-- CreditDebitIndicator: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- TransactionId: long (nullable = true)
 |-- TransactionInformation: string (nullable = true)
 |-- ValueDateTime: string (nullable = true)
 |-- category: string (nullable = true)
 |-- Detailed Category: string (nullable = true)
 |-- AmountInGBP: float (nullable = true)
 |-- Type: string (nullable = true)



In [15]:
pd.DataFrame(TransactionDf.take(5),columns=TransactionDf.columns).transpose()

,0,1,2,3,4
Unnamed: 0,296,76,82,268,269
AccountId,32353546,32353530,32353542,32353522,32353524
Amount,"{""Amount"": ""139.9"", ""Currency"": ""GBP""}","{""Amount"": ""388.5"", ""Currency"": ""GBP""}","{""Amount"": ""50.4"", ""Currency"": ""GBP""}","{""Amount"": ""404"", ""Currency"": ""GBP""}","{""Amount"": ""45"", ""Currency"": ""GBP""}"
BankTransactionCode,"{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }"
BookingDateTime,2016-11-29T20:21:50-08:00,2016-12-05T20:21:50-08:00,2016-12-06T20:21:50-08:00,2017-01-11T20:21:50-08:00,2017-01-24T20:21:50-08:00
CreditDebitIndicator,Credit,Credit,Credit,Credit,Credit
Status,Booked,Booked,Booked,Booked,Booked
TransactionId,100223,100315,100321,100401,100402
TransactionInformation,"/RFB/GMCI STATMT , PCM55CI72578736 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI72684987 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI72706905 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI73243020 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI73446179 , HENNER GMC"
ValueDateTime,2016-11-29T20:21:50-08:00,2016-12-05T20:21:50-08:00,2016-12-06T20:21:50-08:00,2017-01-11T20:21:50-08:00,2017-01-24T20:21:50-08:00


In [16]:
import numpy as np



def BinningData(TransactionDf,bins):
    max_val =TransactionDf.select(TransactionDf.AmountInGBP).rdd.max()[0] 
    min_val =TransactionDf.select(TransactionDf.AmountInGBP).rdd.min()[0] -1  ##??
    custom_bucket_array = np.linspace(min_val, max_val, 4)
    cut_points = list(custom_bucket_array)
    return cut_points

In [17]:
from pyspark.ml.feature import Bucketizer
bucketizer = Bucketizer(splits=BinningData(TransactionDf,4),inputCol="AmountInGBP", outputCol="AmountInGBP_bin")
TransactionDf = bucketizer.transform(TransactionDf)

#TransactionDf.show()

In [18]:
pd.DataFrame(TransactionDf.take(5),columns=TransactionDf.columns).transpose()

,0,1,2,3,4
Unnamed: 0,296,76,82,268,269
AccountId,32353546,32353530,32353542,32353522,32353524
Amount,"{""Amount"": ""139.9"", ""Currency"": ""GBP""}","{""Amount"": ""388.5"", ""Currency"": ""GBP""}","{""Amount"": ""50.4"", ""Currency"": ""GBP""}","{""Amount"": ""404"", ""Currency"": ""GBP""}","{""Amount"": ""45"", ""Currency"": ""GBP""}"
BankTransactionCode,"{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }"
BookingDateTime,2016-11-29T20:21:50-08:00,2016-12-05T20:21:50-08:00,2016-12-06T20:21:50-08:00,2017-01-11T20:21:50-08:00,2017-01-24T20:21:50-08:00
CreditDebitIndicator,Credit,Credit,Credit,Credit,Credit
Status,Booked,Booked,Booked,Booked,Booked
TransactionId,100223,100315,100321,100401,100402
TransactionInformation,"/RFB/GMCI STATMT , PCM55CI72578736 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI72684987 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI72706905 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI73243020 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI73446179 , HENNER GMC"
ValueDateTime,2016-11-29T20:21:50-08:00,2016-12-05T20:21:50-08:00,2016-12-06T20:21:50-08:00,2017-01-11T20:21:50-08:00,2017-01-24T20:21:50-08:00


In [19]:
t = {0:"low", 1: "medium", 2:"high"}
map_bins = udf(lambda x: t[x], StringType())
TransactionDf = TransactionDf.withColumn("AmountInGBP_bin", map_bins("AmountInGBP_bin"))

In [20]:
pd.DataFrame(TransactionDf.take(5),columns=TransactionDf.columns).transpose()

,0,1,2,3,4
Unnamed: 0,296,76,82,268,269
AccountId,32353546,32353530,32353542,32353522,32353524
Amount,"{""Amount"": ""139.9"", ""Currency"": ""GBP""}","{""Amount"": ""388.5"", ""Currency"": ""GBP""}","{""Amount"": ""50.4"", ""Currency"": ""GBP""}","{""Amount"": ""404"", ""Currency"": ""GBP""}","{""Amount"": ""45"", ""Currency"": ""GBP""}"
BankTransactionCode,"{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }"
BookingDateTime,2016-11-29T20:21:50-08:00,2016-12-05T20:21:50-08:00,2016-12-06T20:21:50-08:00,2017-01-11T20:21:50-08:00,2017-01-24T20:21:50-08:00
CreditDebitIndicator,Credit,Credit,Credit,Credit,Credit
Status,Booked,Booked,Booked,Booked,Booked
TransactionId,100223,100315,100321,100401,100402
TransactionInformation,"/RFB/GMCI STATMT , PCM55CI72578736 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI72684987 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI72706905 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI73243020 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI73446179 , HENNER GMC"
ValueDateTime,2016-11-29T20:21:50-08:00,2016-12-05T20:21:50-08:00,2016-12-06T20:21:50-08:00,2017-01-11T20:21:50-08:00,2017-01-24T20:21:50-08:00


In [21]:
# Get Merchant info from Description
def getMerchfromDesc(tranType, tranDesc):
    if tranType == 'POS':
        parsed = tranDesc.split(', ')
        lenPar = len(parsed)
        if parsed[lenPar - 1] == 'REFUND':
            merch = parsed[1:-2]
            return ('').join(merch) + 'REFUND'
        else:
            merch = parsed[1:-1]
            return ('').join(merch)
    else:
        return tranDesc
    
import re    
    
# Clean Merchant info
def cleanMerchant(txt):
    txt = re.sub(r'[^a-zA-Z\b]', ' ', str(txt).upper())
    return " ".join(txt.split())    

In [22]:
parse_merchant = udf(getMerchfromDesc , StringType())
parse_cleanMerchant = udf(cleanMerchant , StringType())
#Type = udf(lambda x: (x.split(",")[0].split(":")[1]).replace('"', '').strip() , StringType())



TransactionDf = TransactionDf.withColumn('Merchant',parse_merchant(TransactionDf['Type'],TransactionDf['TransactionInformation']))
#                              .withColumn('cleanedMerchant',parse_cleanMerchant(TransactionDf['Merchant']))
TransactionDf = TransactionDf.withColumn('cleanedMerchant',parse_cleanMerchant(TransactionDf['Merchant']))

In [23]:
TransactionDf_pd.columns

Index(['Unnamed: 0', 'AccountId', 'Amount', 'BankTransactionCode',
       'BookingDateTime', 'CreditDebitIndicator', 'Status', 'TransactionId',
       'TransactionInformation', 'ValueDateTime', 'category',
       'Detailed Category', 'AmountInGBP', 'Type'],
      dtype='object')

In [24]:
pd.DataFrame(TransactionDf.take(2),columns= TransactionDf.columns).transpose()

,0,1
Unnamed: 0,296,76
AccountId,32353546,32353530
Amount,"{""Amount"": ""139.9"", ""Currency"": ""GBP""}","{""Amount"": ""388.5"", ""Currency"": ""GBP""}"
BankTransactionCode,"{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }"
BookingDateTime,2016-11-29T20:21:50-08:00,2016-12-05T20:21:50-08:00
CreditDebitIndicator,Credit,Credit
Status,Booked,Booked
TransactionId,100223,100315
TransactionInformation,"/RFB/GMCI STATMT , PCM55CI72578736 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI72684987 , HENNER GMC"
ValueDateTime,2016-11-29T20:21:50-08:00,2016-12-05T20:21:50-08:00


In [25]:
#TransactionDf['ConcatCategory'] = TransactionDf['category'] + ', ' + TransactionDf['Detailed Category'] ???



In [26]:
def Predictorcolumn(CreditDebitIndicator,cleanedMerchant,Type,AmountInGBP_bin):
    return CreditDebitIndicator+ ' ' +cleanedMerchant+ ' ' +Type+ ' ' +AmountInGBP_bin

def ConcatCategory(category,Detailed_Category):
    return category+ ','+Detailed_Category

ConcatCategory_udf = udf(ConcatCategory,StringType())

Predictorcolumn_udf = udf(Predictorcolumn,StringType())    
    

In [27]:
TransactionDf = TransactionDf.withColumn('Predictorcolumn',Predictorcolumn_udf(TransactionDf['CreditDebitIndicator'],TransactionDf['cleanedMerchant'],TransactionDf['Type'],TransactionDf['AmountInGBP_bin']))
TransactionDf = TransactionDf.withColumn('ConcatCategory',ConcatCategory_udf(TransactionDf['category'],TransactionDf['Detailed Category']))

In [28]:
pd.DataFrame(TransactionDf.take(2),columns= TransactionDf.columns).transpose()

,0,1
Unnamed: 0,296,76
AccountId,32353546,32353530
Amount,"{""Amount"": ""139.9"", ""Currency"": ""GBP""}","{""Amount"": ""388.5"", ""Currency"": ""GBP""}"
BankTransactionCode,"{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }"
BookingDateTime,2016-11-29T20:21:50-08:00,2016-12-05T20:21:50-08:00
CreditDebitIndicator,Credit,Credit
Status,Booked,Booked
TransactionId,100223,100315
TransactionInformation,"/RFB/GMCI STATMT , PCM55CI72578736 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI72684987 , HENNER GMC"
ValueDateTime,2016-11-29T20:21:50-08:00,2016-12-05T20:21:50-08:00


In [29]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import NGram


In [31]:
tokenizer = Tokenizer(inputCol="Predictorcolumn", outputCol="words")
wordsData = tokenizer.transform(TransactionDf)

In [32]:
#help(tokenizer)
wordsData.head()

Row(Unnamed: 0=296, AccountId=32353546, Amount='{"Amount": "139.9", "Currency": "GBP"}', BankTransactionCode='{"Code":  "CHP"," SubCode": }', BookingDateTime='2016-11-29T20:21:50-08:00', CreditDebitIndicator='Credit', Status='Booked', TransactionId=100223, TransactionInformation='/RFB/GMCI STATMT , PCM55CI72578736 , HENNER GMC', ValueDateTime='2016-11-29T20:21:50-08:00', category='Services', Detailed Category='Health', AmountInGBP=139.89999389648438, Type='CHP', AmountInGBP_bin='low', Merchant='/RFB/GMCI STATMT , PCM55CI72578736 , HENNER GMC', cleanedMerchant='RFB GMCI STATMT PCM CI HENNER GMC', Predictorcolumn='Credit RFB GMCI STATMT PCM CI HENNER GMC CHP low', ConcatCategory='Services,Health', words=['credit', 'rfb', 'gmci', 'statmt', 'pcm', 'ci', 'henner', 'gmc', 'chp', 'low'])

In [33]:
remover = StopWordsRemover(inputCol="words", outputCol="words_no_stop")
removedata = remover.transform(wordsData)

In [34]:
removedata.head()

Row(Unnamed: 0=296, AccountId=32353546, Amount='{"Amount": "139.9", "Currency": "GBP"}', BankTransactionCode='{"Code":  "CHP"," SubCode": }', BookingDateTime='2016-11-29T20:21:50-08:00', CreditDebitIndicator='Credit', Status='Booked', TransactionId=100223, TransactionInformation='/RFB/GMCI STATMT , PCM55CI72578736 , HENNER GMC', ValueDateTime='2016-11-29T20:21:50-08:00', category='Services', Detailed Category='Health', AmountInGBP=139.89999389648438, Type='CHP', AmountInGBP_bin='low', Merchant='/RFB/GMCI STATMT , PCM55CI72578736 , HENNER GMC', cleanedMerchant='RFB GMCI STATMT PCM CI HENNER GMC', Predictorcolumn='Credit RFB GMCI STATMT PCM CI HENNER GMC CHP low', ConcatCategory='Services,Health', words=['credit', 'rfb', 'gmci', 'statmt', 'pcm', 'ci', 'henner', 'gmc', 'chp', 'low'], words_no_stop=['credit', 'rfb', 'gmci', 'statmt', 'pcm', 'ci', 'henner', 'gmc', 'chp', 'low'])

In [ ]:
from pyspark.ml import Pipeline


In [ ]:
#help(pipeline)

In [35]:
ngram = NGram(n=2, inputCol="words_no_stop", outputCol="ngrams_words")

ngramDataFrame = ngram.transform(removedata)
ngramDataFrame.select("ngrams_words").show(truncate=False)


+------------------------------------------------------------------------------------------------+
|ngrams_words                                                                                    |
+------------------------------------------------------------------------------------------------+
|[credit rfb, rfb gmci, gmci statmt, statmt pcm, pcm ci, ci henner, henner gmc, gmc chp, chp low]|
|[credit rfb, rfb gmci, gmci statmt, statmt pcm, pcm ci, ci henner, henner gmc, gmc chp, chp low]|
|[credit rfb, rfb gmci, gmci statmt, statmt pcm, pcm ci, ci henner, henner gmc, gmc chp, chp low]|
|[credit rfb, rfb gmci, gmci statmt, statmt pcm, pcm ci, ci henner, henner gmc, gmc chp, chp low]|
|[credit rfb, rfb gmci, gmci statmt, statmt pcm, pcm ci, ci henner, henner gmc, gmc chp, chp low]|
|[debit rfb, rfb gmci, gmci statmt, statmt pcm, pcm ci, ci henner, henner gmc, gmc chp, chp low] |
|[credit rfb, rfb gmci, gmci statmt, statmt pcm, pcm ci, ci henner, henner gmc, gmc chp, chp low]|
|[debit ni

In [36]:
hashingTF = HashingTF(inputCol="ngrams_words", outputCol="rawFeatures")
featurizedData = hashingTF.transform(ngramDataFrame)
# alternatively, CountVectorizer can also be used to get term frequency vectors
featurizedData.head()


Row(Unnamed: 0=296, AccountId=32353546, Amount='{"Amount": "139.9", "Currency": "GBP"}', BankTransactionCode='{"Code":  "CHP"," SubCode": }', BookingDateTime='2016-11-29T20:21:50-08:00', CreditDebitIndicator='Credit', Status='Booked', TransactionId=100223, TransactionInformation='/RFB/GMCI STATMT , PCM55CI72578736 , HENNER GMC', ValueDateTime='2016-11-29T20:21:50-08:00', category='Services', Detailed Category='Health', AmountInGBP=139.89999389648438, Type='CHP', AmountInGBP_bin='low', Merchant='/RFB/GMCI STATMT , PCM55CI72578736 , HENNER GMC', cleanedMerchant='RFB GMCI STATMT PCM CI HENNER GMC', Predictorcolumn='Credit RFB GMCI STATMT PCM CI HENNER GMC CHP low', ConcatCategory='Services,Health', words=['credit', 'rfb', 'gmci', 'statmt', 'pcm', 'ci', 'henner', 'gmc', 'chp', 'low'], words_no_stop=['credit', 'rfb', 'gmci', 'statmt', 'pcm', 'ci', 'henner', 'gmc', 'chp', 'low'], ngrams_words=['credit rfb', 'rfb gmci', 'gmci statmt', 'statmt pcm', 'pcm ci', 'ci henner', 'henner gmc', 'gmc ch

In [37]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
vectors = idf.fit(featurizedData)
rescaledData = vectors.transform(featurizedData)
rescaledData.head()

Row(Unnamed: 0=296, AccountId=32353546, Amount='{"Amount": "139.9", "Currency": "GBP"}', BankTransactionCode='{"Code":  "CHP"," SubCode": }', BookingDateTime='2016-11-29T20:21:50-08:00', CreditDebitIndicator='Credit', Status='Booked', TransactionId=100223, TransactionInformation='/RFB/GMCI STATMT , PCM55CI72578736 , HENNER GMC', ValueDateTime='2016-11-29T20:21:50-08:00', category='Services', Detailed Category='Health', AmountInGBP=139.89999389648438, Type='CHP', AmountInGBP_bin='low', Merchant='/RFB/GMCI STATMT , PCM55CI72578736 , HENNER GMC', cleanedMerchant='RFB GMCI STATMT PCM CI HENNER GMC', Predictorcolumn='Credit RFB GMCI STATMT PCM CI HENNER GMC CHP low', ConcatCategory='Services,Health', words=['credit', 'rfb', 'gmci', 'statmt', 'pcm', 'ci', 'henner', 'gmc', 'chp', 'low'], words_no_stop=['credit', 'rfb', 'gmci', 'statmt', 'pcm', 'ci', 'henner', 'gmc', 'chp', 'low'], ngrams_words=['credit rfb', 'rfb gmci', 'gmci statmt', 'statmt pcm', 'pcm ci', 'ci henner', 'henner gmc', 'gmc ch

In [ ]:
#import os
#import sys
#path = 'model_idf'
#os.mkdir(path)
#vectors.save('D:/model/model_idf')

In [ ]:
#vectorizer = IDF.load('D:/model/model_idf/')

In [44]:
RandomForest_model = RandomForestClassifier(n_estimators=50, min_samples_split=2, random_state=0)

#RandomForest_model = RandomForestClassifier().setPredictionCol("ConcatCategory").setFeaturesCol("features") 
         
        

#pipeline = (stages=[tokenizer,remover,ngram,hashingTF,idf])
#rfc_model = RandomForest_model.fit(rescaledData, rescaledData['ConcatCategory'])
rfc_model = RandomForest_model.fit(rescaledData.rawFeatures,rescaledData.ConcatCategory)
classifierData = rfc_model.transform(rescaledData)

    
classifierData.head()    
    

ValueError: setting an array element with a sequence.

In [ ]:
help(RandomForestClassifier)

